## best score 0.9

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [4]:
train = pd.read_csv('Data/our_train.csv')
test = pd.read_csv('Data/our_test.csv')
val = pd.read_csv('Data/our_val.csv')
df = pd.read_csv('Data/train.csv')

In [5]:
X_train = train.drop('NObeyesdad', axis=1)
y_train = train['NObeyesdad']
X_test = test.drop('NObeyesdad', axis=1)
y_test = test['NObeyesdad']
X_val = val.drop('NObeyesdad', axis=1)
y_val = val['NObeyesdad']

In [6]:
CAEC_dict = {'no': 0, 'Sometimes': 0.33, 'Frequently': 0.66, 'Always': 1 }
CALC_dict = {'no': 0, 'Sometimes': 0.5, 'Frequently': 1}
X_train['CAEC'] = X_train['CAEC'].map(CAEC_dict)
X_train['CALC'] = X_train['CALC'].map(CALC_dict)
X_test['CAEC'] = X_test['CAEC'].map(CAEC_dict)
X_test['CALC'] = X_test['CALC'].map(CALC_dict)
X_val['CAEC'] = X_val['CAEC'].map(CAEC_dict)
X_val['CALC'] = X_val['CALC'].map(CALC_dict)
cat_cols2 = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'MTRANS']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cat_cols2:
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.fit_transform(X_test[col])
    X_val[col] = le.fit_transform(X_val[col])

y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
y_val = le.fit_transform(y_val)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [9]:
#pierwsze lepsze drzewo
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred1 = rf.predict(X_val)
class_report1 = classification_report(y_val, y_pred1)
print("Classification Report:\n", class_report1)
conf_matrix1 = confusion_matrix(y_val, y_pred1)
print("\nConfusion Matrix:")
print(conf_matrix1)

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.91      0.92       525
           1       0.84      0.89      0.86       624
           2       0.89      0.85      0.87       604
           3       0.96      0.98      0.97       714
           4       0.99      1.00      1.00       837
           5       0.80      0.75      0.77       532
           6       0.78      0.80      0.79       523

    accuracy                           0.89      4359
   macro avg       0.89      0.88      0.88      4359
weighted avg       0.90      0.89      0.89      4359


Confusion Matrix:
[[476  45   0   0   0   3   1]
 [ 26 558   0   0   0  30  10]
 [  2   1 516  24   3  20  38]
 [  0   0  12 697   2   0   3]
 [  1   0   1   1 834   0   0]
 [  1  51  14   0   0 398  68]
 [  0  13  36   4   0  49 421]]


In [12]:
clf = RandomForestClassifier(random_state=42)

RandomForestClassifier( )

param_grid = {
    'n_estimators':  [200, 400, 500, 600],
    'criterion' : ['gini', 'entropy', 'log_loss' ],
    'max_depth': [20, 25, 30],
    'n_jobs':[-1], #to eby uśywalo wszystkich rdzeni procesora 
    'min_samples_split':[ 2, 3,4, 5, 6,7,],

}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END criterion=gini, max_depth=20, min_samples_split=2, n_estimators=200, n_jobs=-1; total time=   5.0s
[CV] END criterion=gini, max_depth=20, min_samples_split=2, n_estimators=200, n_jobs=-1; total time=   5.1s
[CV] END criterion=gini, max_depth=20, min_samples_split=2, n_estimators=200, n_jobs=-1; total time=   5.2s
[CV] END criterion=gini, max_depth=20, min_samples_split=2, n_estimators=400, n_jobs=-1; total time=  10.1s
[CV] END criterion=gini, max_depth=20, min_samples_split=2, n_estimators=400, n_jobs=-1; total time=  10.3s
[CV] END criterion=gini, max_depth=20, min_samples_split=2, n_estimators=400, n_jobs=-1; total time=  10.3s
[CV] END criterion=gini, max_depth=20, min_samples_split=2, n_estimators=500, n_jobs=-1; total time=  13.4s
[CV] END criterion=gini, max_depth=20, min_samples_split=2, n_estimators=500, n_jobs=-1; total time=  13.8s
[CV] END criterion=gini, max_depth=20, min_samples_split=3, n_estimators=

In [15]:
#best
best = RandomForestClassifier(criterion='gini', max_depth=25,min_samples_split=4,n_estimators=500,n_jobs=-1 , random_state=42)
best.fit(X_train, y_train)
y_pred1 = best.predict(X_val)
class_report1 = classification_report(y_val, y_pred1)
print("Classification Report:\n", class_report1)
conf_matrix1 = confusion_matrix(y_val, y_pred1)
print("\nConfusion Matrix:")
print(conf_matrix1)

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.90      0.93       525
           1       0.83      0.91      0.86       624
           2       0.89      0.87      0.88       604
           3       0.96      0.98      0.97       714
           4       0.99      1.00      1.00       837
           5       0.80      0.74      0.77       532
           6       0.79      0.81      0.80       523

    accuracy                           0.90      4359
   macro avg       0.89      0.89      0.89      4359
weighted avg       0.90      0.90      0.90      4359


Confusion Matrix:
[[474  47   0   0   0   3   1]
 [ 18 565   0   0   0  34   7]
 [  2   2 523  21   3  17  36]
 [  0   0  12 697   2   0   3]
 [  1   0   0   1 834   1   0]
 [  1  57  14   0   0 393  67]
 [  0  12  37   4   0  44 426]]
